# 1. Boîte à outils

In [ ]:
import Toolbox as tb

# 2. Diffie-Hellman

Implémenter une communication entre Arielle et Bertrand sous TCP (on
pourra considérer que l'un des deux est un serveur et l'autre un
client) permettant d'appliquer le protocole de Diffie-Hellman. Faites
afficher à chaque participant les étapes du protocole sur sa sortie
standard.

In [ ]:
import User as usr
p, g = 8779, 4157
b = 3242
a = 2682

port = 1024
usr.bertrand(port,g,p,b)
usr.arielle(port,g,p,a)

# 3. L'homme du milieu

Arielle et Bertrand veulent communiquer en utilisant RSA. Implémenter
le serveur de Laurent pour qu'il se fasse passer pour Bertrand auprès
d'Arielle (et inversement). Laurent affichera sur la sortie standard
les messages échangés par Arielle et Bertrand (sous forme déchiffrée).

In [ ]:
import threading
import time
import RSA
import socket as sckt
import Toolbox

def arielle(port):
  def arielle_behavior(port):
    # connexion à Laurent
    socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
    socket.bind(("127.0.0.1", port))
    socket.listen(True)
    print("Arielle : initialized on port {}".format(port))
    print("Arielle : Listening")
    conn, addr = socket.accept()
    print('Arielle : Got new connection from {} at {}'.format(addr[0], addr[1]))
    
    # génération des clés + échange des clés publiques

    (pub1_key, pub2_key), (_,priv_key) = RSA.RSA_gen_keys()
    pub_key1 = 0

    time.sleep(1)
    print("Arielle : Sending {} to {}".format(pub1_key, addr[1]))
    conn.send(str(pub1_key).encode())
    print("Arielle : Sending {} to {}".format(pub2_key, addr[1]))
    conn.send(str(pub2_key).encode())

    pub1_key1 = int(conn.recv(port).decode())
    print("Arielle : Received {} from {}".format(pub1_key1, addr[1]))
    pub2_key1 = int(conn.recv(port).decode())
    print("Arielle : Received {} from {}".format(pub2_key1, addr[1]))
    # communication

    mess = int(conn.recv(port).decode())
    print("Arielle : Received {} from {}".format(mess, addr[1]))
    mess = RSA.RSA_decode(priv_key,pub1_key,mess)
    print("Arielle Received : " + RSA.int_to_text(mess))

    # Fin
    socket.listen(False)
    socket.close()
    print("Arielle : Disconnected")


    # exemple
    # # Arielle attend qu'on lui envoie
    # mess = int(conn.recv(port).decode())
    # print("Arielle : Received {} from {}".format(mess, addr[1]))
    
    # # Arielle recoit
    # mess = ""
    # print("Arielle : Sending {} to {}".format(mess, addr[1]))
    # conn.send(str(mess).encode())

  x = threading.Thread(target=arielle_behavior, args=([port]))
  x.start()
    
def bertrand(port):
  def bertrand_behavior(port):
    # connexion à Bertrand
    socket = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
    socket.bind(("127.0.0.1", port))
    socket.listen(True)
    print("Bertrand : initialized on port {}".format(port))
    print("Bertrand : Listening")
    conn, addr = socket.accept()
    print('Bertrand : Got new connection from {} at {}'.format(addr[0], addr[1]))
    
    # génération des clés + échange des clés publiques

    (pub1_key, pub2_key), (_,priv_key) = RSA.RSA_gen_keys()

    pub1_key1 = 0
    pub2_key1 = 0
    time.sleep(1.5)
    print("Bertrand : Sending {} to {}".format(pub1_key, addr[1]))
    conn.send(str(pub1_key).encode())
    print("Bertrand : Sending {} to {}".format(pub2_key, addr[1]))
    conn.send(str(pub2_key).encode())

    pub1_key1 = int(conn.recv(port).decode())
    print("Bertrand : Received {} from {}".format(pub1_key1, addr[1]))
    pub2_key1 = int(conn.recv(port).decode())
    print("Bertrand : Received {} from {}".format(pub2_key1, addr[1]))

    # communication

    mess = "ca"
    print("Bertrand, sent mess is : ", mess)
    mess = RSA.text_to_int(mess)
    print("ou en int : ",mess)
    mess = RSA.RSA_encode(pub2_key1,pub1_key1,mess)
    print("Bertrand : Sending {} to {}".format(mess, addr[1]))
    conn.send(str(mess).encode())

    # Fin
    socket.listen(False)
    socket.close()
    print("Bertrand : Disconnected")


    # exemple
    # # Bertrand attend qu'on lui envoie
    # mess = int(conn.recv(port).decode())
    # print("Bertrand : Received {} from {}".format(mess, addr[1]))
    
    # # Bertrand recoit
    # mess = ""
    # print("Bertrand : Sending {} to {}".format(mess, addr[1]))
    # conn.send(str(mess).encode())

  x = threading.Thread(target=bertrand_behavior, args=([port]))
  x.start()


def laurent(port1,port2):
  # connexion à Arielle et Bertrand
  # Arielle
  socket1 = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
  try :
    socket1.connect(("127.0.0.1", port1))
    print("Laurent {} : connected".format(socket1.getsockname()[1]))
  except :
    print("Laurent : Connection Error")
    raise Exception
  
  # Bertrand
  socket2 = sckt.socket(sckt.AF_INET, sckt.SOCK_STREAM)
  try :
    socket2.connect(("127.0.0.1", port2))
    print("Laurent {} : connected".format(socket2.getsockname()[1]))
  except :
    print("Laurent : Connection Error")
    raise Exception

  # génération des clés + échange des clés publiques
  
  (self_key_pub1,self_cl_pub2), (_,priv_key) = RSA.RSA_gen_keys()

  pk1_port1 = 0 
  pk2_port1 = 0

  pk1_port2 = 0 
  pk2_port2 = 0

  pk1_port1 = int(socket1.recv(port1).decode())
  print("Laurent 1: Received {} from {}".format(pk1_port1, port1))
  pk2_port1 = int(socket1.recv(port1).decode())
  print("Laurent 2: Received {} from {}".format(pk2_port1, port1))

  pk1_port2 = int(socket2.recv(port2).decode())
  print("Laurent 3: Received {} from {}".format(pk1_port2, port2))
  pk2_port2 = int(socket2.recv(port2).decode())
  print("Laurent 4: Received {} from {}".format(pk2_port2, port2))

  print("Laurent send his keys")
  
  socket1.send(str(self_key_pub1).encode())
  socket2.send(str(self_key_pub1).encode())
  
  time.sleep(0.1)
  socket1.send(str(self_cl_pub2).encode())
  socket2.send(str(self_cl_pub2).encode())

  
  # affichage des messages interceptés

  mess = int(socket2.recv(port2).decode())
  
  print("Laurent gets :",mess," (raw)")
  mess = RSA.RSA_decode(priv_key,self_key_pub1,mess)
  
  print("Laurent gets :",mess," (decoded)")
  mess = RSA.int_to_text(mess)
  print("Laurent gets :",mess," (true message)")

  mess = RSA.text_to_int(mess*2)
  print("laurent send : ",mess)
  mess = RSA.RSA_encode(pk2_port1,pk1_port1,mess)
  print("crypted as :",mess)
  socket1.send(str(mess).encode())

arielle(1024)
bertrand(1025)
laurent(1024,1025)


  

Arielle : initialized on port 1024
Arielle : Listening
Bertrand : initialized on port 1025
Bertrand : Listening
Laurent 64563 : connected
Arielle : Got new connection from 127.0.0.1 at 64563
p: 3151601
q: 4923293
Laurent 64564 : connected
p: 2378569
q: 3792391
Bertrand : Got new connection from 127.0.0.1 at 64564
p: 4925603
q: 4006661
Arielle : Sending 15516255142093 to 64563
Laurent 1: Received 15516255142093 from 1024
Arielle : Sending 7361564295931 to 64563
Laurent 2: Received 7361564295931 from 1024
Bertrand : Sending 19735221441583 to 64564
Bertrand : Sending 19290846015559 to 64564
Laurent 3: Received 19735221441583 from 1025
Laurent 4: Received 19290846015559 from 1025
Laurent send his keys
Arielle : Received 9020463668479 from 64563
Bertrand : Received 9020463668479 from 64564
Arielle : Received 7505401091359 from 64563
Bertrand : Received 7505401091359 from 64564
Bertrand, sent mess is :  test
ou en int :  1930943246
Bertrand : Sending 6919344270306 to 64564
Laurent gets : 691

# 4. Malléabilité

Implémenter Laurent pour que, lorsqu'il intercepte un message chiffré
`c` correspondant au message `m` (qu'il ne peut retrouver), il
transmette à l'autre participant un message `c'` correspondant au
message `2m`.

In [ ]:
def arielle():
    # connexion à Laurent
    
    # génération des clés + échange des clés publiques
    
    # envoi de messages chiffrés (+ affichage)
    return
    
def bertrand():
    # connexion à Laurent
    
    # génération des clés + échange des clés publiques
    
    # déchiffrement des messages reçus (+ affichage)
    return
    
def laurent():
    # connexion à Arielle et Bertrand
    
    # transmission de la clé d'Arielle à Bertrand (et inversement)
   
    # modification et transmission des messages interceptés
    return